In [1]:
import os
from pathlib import Path

import lightning as L
import torch
from torchvision.datasets import EMNIST
from torch.utils.data import Dataset, random_split
from torchvision import datasets
from torchvision.transforms import v2

import utils

In [2]:
# remake these datasets with a new transform to keep them as a tensor

# this dataset is already random rotated
handiso_ds_test = utils.CharDataset("data/handwritten-isolated-english/test", transform=utils.T_tensor, label_transform=utils.class_to_idx)

notmnist_ds = utils.CharDataset("data/notMNIST_small", transform=utils.T_randrottensor, label_transform=utils.class_to_idx)
notmnist_ds_train, notmnist_ds_test = random_split(notmnist_ds, [0.8, 0.2])

stdocr_ds_test = (
    utils.CharDataset("data/standard_ocr_dataset/data/testing_data", transform=utils.T_randrottensor, label_transform=utils.class_to_idx) +
    utils.CharDataset("data/standard_ocr_dataset/data2/testing_data", transform=utils.T_randrottensor, label_transform=utils.class_to_idx)
)

mnist_ds_test_letters = EMNIST(Path(os.getcwd(), "data"), "letters", download=True, train=False, transform=utils.T_randrottensor, target_transform=utils.emnistletter_idx_to_allidx)
mnist_ds_test_digits = EMNIST(Path(os.getcwd(), "data"), "digits", download=True, train=False, transform=utils.T_randrottensor)
mnist_ds_test_orig = mnist_ds_test_letters + mnist_ds_test_digits

mnist_ds_test, _ = random_split(mnist_ds_test_orig, [0.05, 0.95])

consolidated_ds_test = handiso_ds_test + notmnist_ds_test + stdocr_ds_test + mnist_ds_test

Image data/notMNIST_small/A/RGVtb2NyYXRpY2FCb2xkT2xkc3R5bGUgQm9sZC50dGY=.png is not a valid image, skipping
Image data/notMNIST_small/F/Q3Jvc3NvdmVyIEJvbGRPYmxpcXVlLnR0Zg==.png is not a valid image, skipping
Image data/standard_ocr_dataset/data2/testing_data/G/44404.png is not a valid image, skipping


In [8]:
handiso_testloader = torch.utils.data.DataLoader(handiso_ds_test, num_workers=31, batch_size=32)
notmnist_testloader = torch.utils.data.DataLoader(notmnist_ds_test, num_workers=31, batch_size=32)
stdocr_testloader = torch.utils.data.DataLoader(stdocr_ds_test, num_workers=31, batch_size=32)
mnist_testloader = torch.utils.data.DataLoader(mnist_ds_test, num_workers=31, batch_size=32)
testloader = torch.utils.data.DataLoader(consolidated_ds_test, num_workers=31, batch_size=32)

In [4]:
cnnpool = utils.GenericCharCNN.load_from_checkpoint("lightning_logs/cnnres-pool/checkpoints/epoch=19-step=41900.ckpt")
cnnpool.eval()
cnnnopool = utils.GenericCharCNN_nopool.load_from_checkpoint("lightning_logs/cnnres-nopool/checkpoints/epoch=19-step=41900.ckpt")
cnnnopool.eval()
cnnresnet = utils.ResnetTransfered.load_from_checkpoint("lightning_logs/resnet/checkpoints/epoch=19-step=41900.ckpt")
cnnresnet.eval()

In [7]:
trainer = L.Trainer()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/jeyan/ece271bkernel/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


# CNN with pooling

In [9]:
trainer.test(cnnpool, dataloaders=handiso_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 56/56 [00:03<00:00, 15.90it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.3599330484867096
        test_loss            2.18440318107605
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 2.18440318107605, 'test_acc': 0.3599330484867096}]

In [10]:
trainer.test(cnnpool, dataloaders=notmnist_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 117/117 [00:05<00:00, 20.11it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8902243971824646
        test_loss           0.4030971825122833
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.4030971825122833, 'test_acc': 0.8902243971824646}]

In [11]:
trainer.test(cnnpool, dataloaders=stdocr_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 133/133 [00:07<00:00, 17.18it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9333961009979248
        test_loss           0.18268659710884094
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.18268659710884094, 'test_acc': 0.9333961009979248}]

In [12]:
trainer.test(cnnpool, dataloaders=mnist_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 95/95 [00:04<00:00, 19.33it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.807894766330719
        test_loss           0.6707841753959656
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6707841753959656, 'test_acc': 0.807894766330719}]

# CNN without pooling

In [14]:
trainer.test(cnnnopool, dataloaders=handiso_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 56/56 [00:02<00:00, 23.91it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc                 0.6640625
        test_loss           1.0808898210525513
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.0808898210525513, 'test_acc': 0.6640625}]

In [15]:
trainer.test(cnnnopool, dataloaders=notmnist_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 117/117 [00:07<00:00, 14.95it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9204060435295105
        test_loss           0.2823261618614197
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.2823261618614197, 'test_acc': 0.9204060435295105}]

In [16]:
trainer.test(cnnnopool, dataloaders=stdocr_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 133/133 [00:08<00:00, 15.79it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9590491652488708
        test_loss           0.1112338975071907
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.1112338975071907, 'test_acc': 0.9590491652488708}]

In [17]:
trainer.test(cnnnopool, dataloaders=mnist_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 95/95 [00:03<00:00, 31.61it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8325658440589905
        test_loss           0.6113795638084412
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6113795638084412, 'test_acc': 0.8325658440589905}]

# Resnet transfer learning

In [18]:
trainer.test(cnnresnet, dataloaders=handiso_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 56/56 [00:02<00:00, 19.48it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.3063616156578064
        test_loss           2.4301600456237793
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 2.4301600456237793, 'test_acc': 0.3063616156578064}]

In [19]:
trainer.test(cnnresnet, dataloaders=notmnist_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0:  57%|█████▋    | 67/117 [00:06<00:04, 10.73it/s]

Testing DataLoader 0: 100%|██████████| 117/117 [00:07<00:00, 15.36it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8039530515670776
        test_loss           0.6919888854026794
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.6919888854026794, 'test_acc': 0.8039530515670776}]

In [20]:
trainer.test(cnnresnet, dataloaders=stdocr_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 133/133 [00:08<00:00, 14.89it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.8420804738998413
        test_loss            0.536267101764679
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.536267101764679, 'test_acc': 0.8420804738998413}]

In [21]:
trainer.test(cnnresnet, dataloaders=mnist_testloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 95/95 [00:05<00:00, 16.58it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6796053051948547
        test_loss           1.1777197122573853
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.1777197122573853, 'test_acc': 0.6796053051948547}]